
Exported data out of Account Management Center at [https://www.garmin.com/en-US/account/datamanagement/](https://www.garmin.com/en-US/account/datamanagement/) on 2021-06-20

In [1]:
import os
import json
import pandas as pd
from glob import glob

In [2]:
basepath_repo = os.path.dirname(os.getcwd())
basepath_garmin = '/mnt/g/My Drive/Health/Garmin_20210620/'
account_email = 'meghan.thommes@gmail.com'

# Functions

In [3]:
def read_json(fname:str):
    '''
    read json from file
    REQUIRED INPUTS
        fname (str): full path to file
    RETURNS
        data (dict*): the read json file, presumably a dict
    '''
    
    # open file
    f = open(fname)
    
    # read file
    data = json.load(f)
    
    # close file
    f.close()
    
    return data

In [4]:
def dict_to_df(data_dict:dict):
    '''
    convert a list of dicts to a dataframe
    REQUIRED INPUTS
        data_dict (dict): list of dictionaries to be converted to a dataframe,
                          where each dictionary should have the same keys
    RETURNS
        data_df (pd.DataFrame): pandas dataframe of all elements in data_dict
    '''
    
    # get column names
    col_names = list(data_dict[0].keys())
    
    # loop through dict and convert to dataframe
    data_df = pd.concat(
        [pd.DataFrame(elem, index = [0], columns = col_names) for elem in data_dict],
        ignore_index = True,
    )
    
    return data_df

In [5]:
def read_sleep_json(fname:str):
    '''
    convert json containing sleep data to pandas DataFrame
    REQUIRED INPUTS
        fname (str): full path to json file of Garmin sleep data
    RETURNS
        sleep_df (pd.DataFrame): pandas dataframe of Garmin sleep data
    '''
    
    # load data
    sleep_dict = read_json(fname)
    
    # convert to dataframe
    sleep_df = dict_to_df(sleep_dict)

    return sleep_df

# Summarized Activities

In [6]:
ftype = 'summarizedActivities'
# filename
fname = os.path.join(
    basepath_garmin,
    'DI_CONNECT/DI-Connect-Fitness/' + account_email + '_0_' + ftype + '.json,DI-Connect-Fitness',
    account_email + '_0_' + ftype + '.json'
)
# load data
activities_data = read_json(fname)
# loads as a list, not a dict, so get first element
activities_dict = activities_data[0][ftype + 'Export']
# convert to dataframe
activities_df = dict_to_df(activities_dict)

activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   activityId            909 non-null    int64  
 1   uuidMsb               909 non-null    int64  
 2   uuidLsb               909 non-null    int64  
 3   name                  909 non-null    object 
 4   activityType          909 non-null    object 
 5   userProfileId         909 non-null    int64  
 6   timeZoneId            909 non-null    int64  
 7   beginTimestamp        909 non-null    int64  
 8   eventTypeId           909 non-null    int64  
 9   rule                  909 non-null    object 
 10  sportType             909 non-null    object 
 11  startTimeGmt          909 non-null    float64
 12  startTimeLocal        909 non-null    float64
 13  duration              909 non-null    float64
 14  distance              907 non-null    object 
 15  elevationGain         7

## Save to CSV

In [7]:
fname_save = os.path.join(
    basepath_repo,
    'data',
    'processed',
    ftype + '.csv'
)

activities_df.to_csv(
    fname_save,
    index = False,
)

# Sleep Data

In [8]:
ftype = 'sleepData'
# filename
fpath = os.path.join(
    basepath_garmin,
    'DI_CONNECT/DI-Connect-Wellness/'
)

folders = glob(fpath + '/*_' + ftype + '.json*')
# each folder contains a single json
fnames = [glob(folder + '/*.json')[0] for folder in folders]

# concat all files into single dataframe
sleep_df = pd.concat(
    [read_sleep_json(fname) for fname in fnames],
    ignore_index = True,
)

sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   sleepStartTimestampGMT       830 non-null    object
 1   sleepEndTimestampGMT         830 non-null    object
 2   calendarDate                 830 non-null    object
 3   sleepWindowConfirmationType  830 non-null    object
 4   deepSleepSeconds             633 non-null    object
 5   lightSleepSeconds            633 non-null    object
 6   remSleepSeconds              513 non-null    object
 7   awakeSleepSeconds            633 non-null    object
 8   unmeasurableSeconds          633 non-null    object
 9   retro                        830 non-null    bool  
 10  averageRespiration           95 non-null     object
 11  lowestRespiration            95 non-null     object
 12  highestRespiration           95 non-null     object
dtypes: bool(1), object(12)
memory usage

## Save to CSV

In [9]:
fname_save = os.path.join(
    basepath_repo,
    'data',
    'processed',
    ftype + '.csv'
)

sleep_df.to_csv(
    fname_save,
    index = False,
)